In [14]:
# comando de Jupyter para que las imagenes se muestren automaticamente 
%matplotlib inline

import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Flatten,Conv2D,Dense,MaxPooling2D,InputLayer,Activation,GlobalAveragePooling2D, Input, Concatenate, Flatten,BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

# Carga Datos

In [15]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [16]:
# summarize loaded dataset
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [17]:
n,h,w,c = x_train.shape
classes=int(y_train.max()+1)

# Aumento de clases

In [19]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [20]:
# Aca hacerlo por canal
for i in range(c):
    x_train = (x_train - x_train.mean()) / x_train.std()
    x_test = (x_test - x_test.mean()) / x_test.std()

In [21]:
generator = datagen.flow(x_train, y_train, batch_size=32)

In [22]:
input_shape = (32, 32, 3)

## Modelo 1

In [28]:
es = EarlyStopping(monitor='val_acc', mode='min', patience=5, verbose=1)

model = Sequential()
model.add(Conv2D(64, kernel_size = 7, activation='relu', input_shape=(input_shape), strides = 1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(Flatten())
model.add(Dense(classes, activation='softmax'))

model.compile(
  optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,amsgrad=False), 
  loss='sparse_categorical_crossentropy', 
  # metricas para ir calculando en cada iteracion o batch 
  # Agregamos el accuracy del modelo
  metrics=['accuracy'], 
)

In [29]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 64)        9472      
_________________________________________________________________
batch_normalization_4 (Batch (None, 26, 26, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                108170    
Total params: 117,898
Trainable params: 117,770
Non-trainable params: 128
_________________________________________________________________
None


In [30]:
model.fit_generator(generator, steps_per_epoch=len(generator)/32, epochs=100, validation_data=(x_test,y_test), callbacks=[es])

Epoch 1/100


C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


49/48 [==============================] - 12s 238ms/step - loss: 3.8247 - acc: 0.2124 - val_loss: 3.8659 - val_acc: 0.2420
Epoch 2/100
49/48 [==============================] - 13s 258ms/step - loss: 3.4741 - acc: 0.2366 - val_loss: 3.8660 - val_acc: 0.2483
Epoch 3/100
49/48 [==============================] - 13s 265ms/step - loss: 3.3371 - acc: 0.2628 - val_loss: 3.3827 - val_acc: 0.2940
Epoch 4/100
49/48 [==============================] - 12s 247ms/step - loss: 3.3843 - acc: 0.2545 - val_loss: 3.3925 - val_acc: 0.2698
Epoch 5/100
49/48 [==============================] - 12s 243ms/step - loss: 3.3622 - acc: 0.2672 - val_loss: 3.2833 - val_acc: 0.3039
Epoch 6/100
49/48 [==============================] - 12s 243ms/step - loss: 3.0557 - acc: 0.2800 - val_loss: 2.4940 - val_acc: 0.3558
Epoch 00006: early stopping


In [31]:
test_generator = datagen.flow(x_test,y_test,batch_size=32)
model.evaluate_generator(test_generator)

[2.776865305328369, 0.3173]

## Modelo 2

In [32]:
conv_filters=32
dense_filters=64

model = Sequential()
model.add(Conv2D(conv_filters,(3,3),activation=None,input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(dense_filters,activation=None))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(classes,activation="softmax"))

model.compile(
  optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,amsgrad=False), 
  loss='sparse_categorical_crossentropy', 
  # metricas para ir calculando en cada iteracion o batch 
  # Agregamos el accuracy del modelo
  metrics=['accuracy'], 
)

In [33]:
model.fit_generator(generator, steps_per_epoch=len(generator)/32, epochs=100, validation_data=(x_test,y_test), callbacks=[es])

Epoch 1/100


C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


49/48 [==============================] - 9s 192ms/step - loss: 2.1532 - acc: 0.2398 - val_loss: 1.9898 - val_acc: 0.2908
Epoch 2/100
49/48 [==============================] - 9s 176ms/step - loss: 1.9051 - acc: 0.3119 - val_loss: 1.8571 - val_acc: 0.3457
Epoch 3/100
49/48 [==============================] - 9s 174ms/step - loss: 1.8611 - acc: 0.3316 - val_loss: 1.8016 - val_acc: 0.3519
Epoch 4/100
49/48 [==============================] - 9s 175ms/step - loss: 1.8126 - acc: 0.3297 - val_loss: 1.6974 - val_acc: 0.3831
Epoch 5/100
49/48 [==============================] - 9s 175ms/step - loss: 1.7820 - acc: 0.3610 - val_loss: 1.7199 - val_acc: 0.3868
Epoch 6/100
49/48 [==============================] - 8s 172ms/step - loss: 1.7142 - acc: 0.4018 - val_loss: 1.6460 - val_acc: 0.4192
Epoch 00006: early stopping


In [34]:
test_generator = datagen.flow(x_test,y_test,batch_size=32)
model.evaluate_generator(test_generator)

[1.720861089515686, 0.3847]